In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from riotwatcher import LolWatcher, ApiError
from datetime import datetime, timedelta
import time

In [183]:
lol_watcher = LolWatcher('RGAPI-ff9da448-f0de-44ce-84ef-46e388e74a72')
my_region = 'euw1'


In [95]:
def liste_player_all_ranks():
    list_finale = []
    list_finale = pd.DataFrame(list_finale, columns=['puuid'])
    random_page = np.random.randint(1, 10)
    for tier in ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']:
        for division in ['I', 'II', 'III', 'IV']:
            try:
                listloc = lol_watcher.league.entries(my_region,
                                                   queue='RANKED_SOLO_5x5',
                                                   tier=tier,
                                                   division=division,
                                                   page=random_page)
                listloc = pd.DataFrame(listloc)
                listloc = pd.DataFrame(listloc['puuid'].drop_duplicates().reset_index(drop=True))
                listloc.columns = ['puuid']
                list_finale = pd.concat([list_finale, listloc], ignore_index=True)
                print(f"Loaded {len(listloc)} players from {tier} {division}")
            except ApiError as err:
                print(f"Error fetching data for {tier} {division}: {err}")
    
    try:
        listlocchall = lol_watcher.league.challenger_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocchall = pd.DataFrame(listlocchall)
        listchallpuuid = listlocchall['entries'].apply(lambda x: x['puuid'])
        listlocchall = pd.DataFrame(listchallpuuid).drop_duplicates().reset_index(drop=True)
        listlocchall.columns = ['puuid']
        listlocchall = pd.DataFrame(listlocchall['puuid'].sample(205).drop_duplicates().reset_index(drop=True))
        listlocchall.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocchall], ignore_index=True)
        print(f"Loaded {len(listlocchall)} players from challenger")
    except ApiError as err:
        print(f"Error fetching data for challenger: {err}")
    
    try:
        listlocmaster = lol_watcher.league.masters_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocmaster = pd.DataFrame(listlocmaster)
        listmasterpuuid = listlocmaster['entries'].apply(lambda x: x['puuid'])
        listlocmaster = pd.DataFrame(listmasterpuuid).drop_duplicates().reset_index(drop=True)
        listlocmaster.columns = ['puuid']
        listlocmaster = pd.DataFrame(listlocmaster['puuid'].sample(410).drop_duplicates().reset_index(drop=True))
        listlocmaster.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocmaster], ignore_index=True)
        print(f"Loaded {len(listlocmaster)} players from master")
    except ApiError as err:
        print(f"Error fetching data for master: {err}")

    try:
        listlocgrandmaster = lol_watcher.league.grandmaster_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocgrandmaster = pd.DataFrame(listlocgrandmaster)
        listgrandmasterpuuid = listlocgrandmaster['entries'].apply(lambda x: x['puuid'])
        listlocgrandmaster = pd.DataFrame(listgrandmasterpuuid).drop_duplicates().reset_index(drop=True)
        listlocgrandmaster.columns = ['puuid']
        listlocgrandmaster = pd.DataFrame(listlocgrandmaster['puuid'].sample(205).drop_duplicates().reset_index(drop=True))
        listlocgrandmaster.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocgrandmaster], ignore_index=True)
        print(f"Loaded {len(listlocgrandmaster)} players from grand master")
    except ApiError as err:
        print(f"Error fetching data for grand master: {err}")
    
    list_finale = list_finale.drop_duplicates().reset_index(drop=True)
    return list_finale


In [135]:
df_test = liste_player_all_ranks()

Loaded 205 players from IRON I
Loaded 205 players from IRON II
Loaded 205 players from IRON III
Loaded 205 players from IRON IV
Loaded 205 players from BRONZE I
Loaded 205 players from BRONZE II
Loaded 205 players from BRONZE III
Loaded 205 players from BRONZE IV
Loaded 205 players from SILVER I
Loaded 205 players from SILVER II
Loaded 205 players from SILVER III
Loaded 205 players from SILVER IV
Loaded 205 players from GOLD I
Loaded 205 players from GOLD II
Loaded 205 players from GOLD III
Loaded 205 players from GOLD IV
Loaded 205 players from PLATINUM I
Loaded 205 players from PLATINUM II
Loaded 205 players from PLATINUM III
Loaded 205 players from PLATINUM IV
Loaded 205 players from DIAMOND I
Loaded 205 players from DIAMOND II
Loaded 205 players from DIAMOND III
Loaded 205 players from DIAMOND IV
Loaded 205 players from challenger
Loaded 410 players from master
Loaded 205 players from grand master


In [145]:
df_test.head()

,puuid
0,UEgCPIQwqomxJO8fr__NA6SPtASX-gyzqynrd5T7FE8z0a...
1,sSr0hDquRqOlGeO5oqR24CXZ9LeSLb_B8gXhdMQlAxGCG2...
2,O435vOI81ZxLb2cqxi_q-0N2y_b_ge4KOEkxhfdpj47VsC...
3,eKAxixfYcOfBmGc9o0wR-IjQ8FlnfwSfB15idqKX_xGyDb...
4,DxmmF5Ye378Mp_GkQK7CWg0pbI6yrKW9qHIRNqAn0eHkyz...


In [ ]:
def last_ranked_games_by_puuid(puuidloc, nbgames):
    try:
        matchlist = lol_watcher.match.matchlist_by_puuid(region = my_region,
                                                         puuid = puuidloc,
                                                         queue=420, # RANKED_SOLO_5x5
                                                         type='ranked', #optional
                                                         count=nbgames)
        return matchlist  
    except ApiError as err:
        print(f"Error fetching match list for {puuidloc}: {err}")
    return None

In [158]:
def create_last_games_df(puuid_df, nbgames):
    last_games = []
    for puuid in puuid_df['puuid']:
        last_games_id = last_ranked_games_by_puuid(puuid, nbgames)
        if last_games_id:
            last_games.append(last_games_id)
        else:
            last_games.append(None)
    last_games_df = pd.DataFrame(last_games)
    last_games_df = last_games_df.dropna().reset_index(drop=True)
    last_games_df = last_games_df.rename(columns={0: 'game_id'})
    last_games_df = last_games_df.set_index('game_id')
    return last_games_df

In [171]:

def get_match_players_roles(region, game_id):
    match_info = lol_watcher.match.by_id(region, game_id)
    participants = match_info['info']['participants']

    # Map teamId → side
    side_map = {100: "BlueSide", 200: "RedSide"}

    # Initialize dict for this game
    game_data = {"gameId": game_id}

    for p in participants:
        side = side_map[p["teamId"]]

        # Lane/role from Riot
        lane = p["lane"]
        role = p["role"]

        # Normalize into standard LoL positions
        if lane == "TOP":
            position = "Top"
        elif lane == "JUNGLE":
            position = "Jungle"
        elif lane == "MIDDLE":
            position = "Mid"
        elif lane == "BOTTOM":
            position = "ADC" if role == "DUO_CARRY" else "Support"
        elif lane == "UTILITY":
            position = "Support"
        else:
            position = lane  # fallback (in case Riot returns weird stuff)

        # Build column names
        puuid_col = f"{side}{position}_puuid"

        # Fill the dict
        game_data[puuid_col] = p["puuid"]
    
    # Add victorious side
    teams_info = match_info['info']['teams']
    if teams_info[0]['win']:   # teamId 100 → BlueSide
        game_data["TeamVictorious"] = "BlueSide"
    else:
        game_data["TeamVictorious"] = "RedSide"

    return pd.DataFrame([game_data])


In [177]:
def build_player_features(region, puuid, reference_game_id, games_used):
    """
    Build player features based on their historical games up to reference_game_id.
    Only considers games that occurred BEFORE the reference game to avoid data leakage.
    season_games: number of recent games to consider (can be limited by API).
    """ 
    try:
        # 1️⃣ Récupérer le match de référence pour obtenir son timestamp et le rôle
        reference_match = lol_watcher.match.by_id(region, reference_game_id)
        reference_timestamp = reference_match['info']['gameCreation']
        
        # Déterminer le rôle du joueur dans le match de référence
        reference_participant = None
        for p in reference_match['info']['participants']:
            if p['puuid'] == puuid:
                reference_participant = p
                break
        
        if reference_participant is None:
            print(f"Player {puuid} not found in reference game {reference_game_id}")
            return None
            
        # Normaliser la position de référence
        lane = reference_participant['lane']
        role = reference_participant['role']
        
        if lane == "TOP":
            reference_position = "Top"
        elif lane == "JUNGLE":
            reference_position = "Jungle"
        elif lane == "MIDDLE":
            reference_position = "Mid"
        elif lane == "BOTTOM":
            reference_position = "ADC" if role == "DUO_CARRY" else "Support"
        elif lane == "UTILITY":
            reference_position = "Support"
        else:
            reference_position = lane
            
    except ApiError as err:
        print(f"Error fetching reference match {reference_game_id}: {err}")
        return None
    
    try:
        # 2️⃣ Récupérer l'historique de la saison pour le joueur
        matchlist = lol_watcher.match.matchlist_by_puuid(
            region=region,
            puuid=puuid,
            type='ranked',
            queue=420,
            count=games_used
        )
        
        if not matchlist:
            print(f"No matches found for player {puuid}")
            return None
            
    except ApiError as err:
        print(f"Error fetching match list for {puuid}: {err}")
        return None

    # 3️⃣ Récupérer les infos détaillées de chaque game avec gestion d'erreurs
    # Filtrer seulement les games AVANT le match de référence
    games_info = []
    for i, game_id in enumerate(matchlist):
        try:
            match = lol_watcher.match.by_id(region, game_id)
            
            # Ne garder que les games antérieures au match de référence
            if match['info']['gameCreation'] < reference_timestamp:
                games_info.append(match)
            
            # Rate limiting - pause entre les appels API
            if i % 10 == 9:  # Pause tous les 10 appels
                time.sleep(1)
                
        except ApiError as err:
            print(f"Error fetching match {game_id}: {err}")
            continue  # Ignorer cette game et continuer
    
    if not games_info:
        print(f"No valid historical match data found for player {puuid} before reference game")
        return None
    
    # 4️⃣ Extraire features par game
    records = []
    for m in games_info:
        try:
            # Trouver le participant correspondant au puuid
            participant = None
            for p in m['info']['participants']:
                if p['puuid'] == puuid:
                    participant = p
                    break
            
            if participant is None:
                continue  # Skip cette game si le joueur n'est pas trouvé
            
            # Normaliser le lane comme dans get_match_players_roles
            lane = participant['lane']
            role = participant['role']
            
            if lane == "TOP":
                position = "Top"
            elif lane == "JUNGLE":
                position = "Jungle"
            elif lane == "MIDDLE":
                position = "Mid"
            elif lane == "BOTTOM":
                position = "ADC" if role == "DUO_CARRY" else "Support"
            elif lane == "UTILITY":
                position = "Support"
            else:
                position = lane
            
            records.append({
                'gameId': m['metadata']['matchId'],
                'timestamp': m['info']['gameCreation'] / 1000,  # convert ms → s
                'win': 1 if participant['win'] else 0,
                'lane': lane,
                'position': position,  # Position normalisée
                'role': role,
                'teamId': participant['teamId'],
                'gameDuration': m['info']['gameDuration'],
            })
            
        except Exception as e:
            print(f"Error processing match {m.get('metadata', {}).get('matchId', 'unknown')}: {e}")
            continue

    if not records:
        print(f"No valid game records found for player {puuid}")
        return None
        
    df = pd.DataFrame(records)
    df['date'] = pd.to_datetime(df['timestamp'], unit='s')
    
    # Trier par timestamp pour avoir les plus récents en dernier
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    # 5️⃣ Features globales avec vérifications
    try:
        main_position = reference_position  # Utiliser la position du match de référence
        total_games_position = len(df[df['position'] == main_position])
        winrate_position = df[df['position'] == main_position]['win'].mean() if total_games_position > 0 else 0.5
        
        total_games = len(df)
        winrate_total = df['win'].mean() if total_games > 0 else 0.5
        
        # 6️⃣ WR derniers N matchs
        wr_30 = df['win'].tail(30).mean() if len(df) >= 30 else df['win'].mean()
        wr_20 = df['win'].tail(20).mean() if len(df) >= 20 else df['win'].mean()
        wr_10 = df['win'].tail(10).mean() if len(df) >= 10 else df['win'].mean()
        wr_5  = df['win'].tail(5).mean() if len(df) >= 5 else df['win'].mean()
        
        # 7️⃣ Streak
        streak = 0
        if len(df) > 0:
            last_result = None
            for w in df['win'][::-1]:  # dernier match en premier
                if last_result is None:
                    last_result = w
                    streak = 1 if w == 1 else -1
                elif (streak > 0 and w == 1) or (streak < 0 and w == 0):
                    streak += 1 if w == 1 else -1
                else:
                    break
        
        # 8️⃣ Games et WR sur les derniers 7/3/1 jours (basé sur la date du match de référence)
        now = pd.to_datetime(reference_timestamp / 1000, unit='s')  # Date du match de référence
        games_7d = df[df['date'] >= now - timedelta(days=7)]
        games_3d = df[df['date'] >= now - timedelta(days=3)]
        games_1d = df[df['date'] >= now - timedelta(days=1)]
        
        wr_7d = games_7d['win'].mean() if len(games_7d) > 0 else winrate_total
        wr_3d = games_3d['win'].mean() if len(games_3d) > 0 else winrate_total
        wr_1d = games_1d['win'].mean() if len(games_1d) > 0 else winrate_total
        
        games_7d_count = len(games_7d)
        games_3d_count = len(games_3d)
        games_1d_count = len(games_1d)
        
        # 9️⃣ Heure de la game de référence
        game_hour = now.hour  # Heure du match de référence
        hour_games = df[df['date'].dt.hour == game_hour]
        wr_hour = hour_games['win'].mean() if len(hour_games) > 0 else winrate_total
        
        # 🔟 Construire le dict final
        features = {
            'puuid': puuid,
            'main_position': main_position,
            'total_games_position': total_games_position,
            'winrate_position': winrate_position,
            'total_games': total_games,
            'winrate_total': winrate_total,
            'wr_30': wr_30,
            'wr_20': wr_20,
            'wr_10': wr_10,
            'wr_5': wr_5,
            'streak': streak,
            'games_7d': games_7d_count,
            'games_3d': games_3d_count,
            'games_1d': games_1d_count,
            'wr_7d': wr_7d,
            'wr_3d': wr_3d,
            'wr_1d': wr_1d,
            'game_hour': game_hour,
            'wr_hour': wr_hour
        }
        
        return features
        
    except Exception as e:
        print(f"Error calculating features for player {puuid}: {e}")
        return None